In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from PIL import Image 
import cv2
import os
import collections
import keras
from keras.utils import to_categorical
import random
from random import shuffle
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.model_selection import  train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')
import os
print(os.listdir("../input"))
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

Using TensorFlow backend.


['fruits-360_dataset']
fruits-360_dataset



In [2]:
indi = 50
epochs = 8
img_size = 75
np.random.seed(4444)

In [3]:
def fitness(g,n):
    c1 = g[1]
    c2 = 2*c1
    c3 = 2*c2
    c4 = 2*c3
    aug = g[2]
    d1 = 8*c4
    d2 = 4*c4

    d_out_c = g[3]
    d_out_d = d_out_c*2
    lr1 = g[4]
    epochs=g[5]
    batch_size=g[6]
    activ = g[7]
    model=Sequential()

    model.add(Conv2D(filters=c1,kernel_size=(5,5),padding="Same",activation=activ,input_shape=(img_size,img_size,1)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(d_out_c))

    model.add(Conv2D(filters=c2,kernel_size=(5,5),padding="Same",activation=activ))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(d_out_c))

    model.add(Conv2D(filters=c3,kernel_size=(5,5),padding="Same",activation=activ))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(d_out_c))

    model.add(Conv2D(filters=c4,kernel_size=(5,5),padding="Same",activation=activ))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(d_out_c))

    model.add(Flatten())

    model.add(Dense(d1,activation=activ))
    model.add(Dropout(d_out_d))

    model.add(Dense(d2,activation=activ))
    model.add(Dropout(d_out_d))

    model.add(Dense(n,activation="softmax"))

    optimizer=Adam(lr=lr1,beta_1=0.9,beta_2=0.999)
    model.compile(optimizer=optimizer,loss="binary_crossentropy",metrics=["accuracy"])


    datagen=ImageDataGenerator(featurewise_center=False, 
                               samplewise_center=False,  
                               featurewise_std_normalization=False, 
                               samplewise_std_normalization=False,  
                               zca_whitening=False,  
                               rotation_range=aug,  
                               zoom_range=aug,      
                               width_shift_range=aug, 
                               height_shift_range=aug,
                               horizontal_flip=False,  
                               vertical_flip=False,
                               )
    datagen.fit(x_train)

#     history=
    model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),
              epochs=epochs,
              validation_data=(x_val,y_val),
              steps_per_epoch=x_train.shape[0]//batch_size
             )
    y_pred=model.predict(x_val)
    y_pred_classes=np.argmax(y_pred,axis=1)
    y_true=np.argmax(y_val,axis=1)
    n = 0;
    d = 0;
    for i in range(len(y_true)):
        if(y_true[i] == y_pred_classes[i]):
            n += 1
        d += 1
    val_accu=n*100/d
    print(val_accu, "Percent validation accuracy")

    y_pred2=model.predict(X_test)
    y_pred_classes2=np.argmax(y_pred2,axis=1)
    y_true2=np.argmax(Y_test,axis=1)

    n = 0;
    d = 0;
    for i in range(len(y_true)):
        if(y_true2[i] == y_pred_classes2[i]):
            n += 1
        d += 1
    test_accu=n*100/d
    print(test_accu, "Percent test accuracy")
    
    
    cost = test_accu*(0.9) + val_accu*(0.1)
    return cost

In [4]:
def load_data(indi):
    data_path1="../input/fruits-360_dataset/fruits-360/Training/"
    classes = []
    ind = 1
    aa = []
    aa.append('Apple Red 1')
    aa.append('Potato Sweet')
    # aa.append('Pear')
    aa.append('Carambula')
    aa.append('Pear Red')
    aa.append('Apple Golden 1')
    aa.append('Banana')
    for folder_name in os.listdir(data_path1):
        if(folder_name in aa):
            pass
        else:
            classes.append(folder_name)
            ind+=1
        if(ind>indi):
            break

    all_arrays=[]

    ind=1
    print("Reading training data: ")
    for i in classes:
        path=os.path.join(data_path1,i)
        class_num=classes.index(i)
        if(ind==1 or ind%int(indi/10)==0):
            print(ind, "Out of ",len(classes), " is done")
        ind+=1
        for img in os.listdir(path):
            img_array=cv2.imread(os.path.join(path,img),
                                 cv2.IMREAD_GRAYSCALE)
            img_array=cv2.resize(img_array,(img_size,img_size))
            all_arrays.append([img_array,class_num])
        if ind>indi:
            break

    data_path2="../input/fruits-360_dataset/fruits-360/Test/"
    classes2 = []
    ind = 1
    for folder_name in os.listdir(data_path2):
        if(folder_name in aa):
            pass
        else:
            classes2.append(folder_name)
            ind+=1
        if ind>indi:
            break
    print(len(classes2), "fruits for Testing ")

    all_arrays2 = []

    ind = 1
    print("Reading training data: ")
    for i in classes2:
        path=os.path.join(data_path2,i)
        class_num2=classes2.index(i)
        if(ind==1 or ind%int(indi/10)==0):
            print(ind, "Out of ",len(classes)," is done")
        ind+=1
        for img in os.listdir(path):
            img_array2=cv2.imread(os.path.join(path,img),
                                 cv2.IMREAD_GRAYSCALE)
            img_array2=cv2.resize(img_array2,(img_size,img_size))
            all_arrays2.append([img_array2,class_num2])
        if ind>indi:
            break

    fruits_array_train=[]
    for features,label in all_arrays:
        fruits_array_train.append(features)


    shuffle(all_arrays)

    X_train=[]
    Y_train=[]
    for features,label in all_arrays:
        X_train.append(features)
        Y_train.append(label)
    X_train=np.array(X_train)

    import random
    random.shuffle(all_arrays2)

    X_test=[]
    Y_test=[]
    for features,label in all_arrays2:
        X_test.append(features)
        Y_test.append(label)
    X_test=np.array(X_test)

    X_train=X_train.reshape(-1,img_size,img_size,1)
    X_train=X_train/255
    X_test=X_test.reshape(-1,img_size,img_size,1)
    X_test=X_test/255
    print("shape of X_train= ",X_train.shape)
    print("shape of X_test=  ",X_test.shape)


    Y_train=to_categorical(Y_train,num_classes=len(classes))
    Y_test=to_categorical(Y_test,num_classes=len(classes))

    x_train,x_val,y_train,y_val=train_test_split(X_train,Y_train,test_size=0.15,random_state=44)
    return x_train,x_val,y_train,y_val,X_test,Y_test

In [5]:
x_train,x_val,y_train,y_val,X_test,Y_test=load_data(indi)

Reading training data: 
1 Out of  50  is done
5 Out of  50  is done
10 Out of  50  is done
15 Out of  50  is done
20 Out of  50  is done
25 Out of  50  is done
30 Out of  50  is done
35 Out of  50  is done
40 Out of  50  is done
45 Out of  50  is done
50 Out of  50  is done
50 fruits for Testing 
Reading training data: 
1 Out of  50  is done
5 Out of  50  is done
10 Out of  50  is done
15 Out of  50  is done
20 Out of  50  is done
25 Out of  50  is done
30 Out of  50  is done
35 Out of  50  is done
40 Out of  50  is done
45 Out of  50  is done
50 Out of  50  is done
shape of X_train=  (25217, 75, 75, 1)
shape of X_test=   (8590, 75, 75, 1)


In [6]:
def populate(pop):
    genes = []
    for _ in range(pop):
        gname = ''
        gname+=str(_)
        c1 = 2**(random.randint(3,5))
        aug = 0
        d_out_c = random.randint(10,35)/100
        lr1 = random.randint(1,20)/10000
        batch_size = 2**(random.randint(4,7))
        activ = ['sigmoid','relu']
        # np.random.seed(4444)
        accur = -1
        genes.append([gname,c1,aug,d_out_c,lr1,epochs,batch_size,activ[1],accur])
    return genes

def crossover(gg1,gg2):
    g1 = gg1
    g2 = gg2
    cross_p = int(g1[8]/2)
    g1[0],g2[0] = g1[0]+str(random.randint(1,9)),g2[0]+str(random.randint(1,9))
    for i in range(1,len(g1)):
        r = random.randint(0,100)
        if(r<cross_p):
            g1[i],g2[i] = g2[i],g1[i]
    return g1,g2

def mutation(genes):
    mut = 20
    for i in genes:
        r1 = random.randint(3,int(len(genes)/2))
        r = random.randint(3,len(genes)-1)
        rmut = random.randint(1,100)
        if(rmut<mut):
            rg = random.randint(1,len(genes[r1])-1)
            genes[r1][rg] = genes[r][rg]
            
def cross_over(genes):
    for i in range(3,int(len(genes)/2)):
        a = random.randint(1,4)
        g1,g2 = crossover(genes[len(genes)-a],genes[i])
        genes.append(g1)
        genes.append(g2)

def natural_selection(genes):
    n = len(genes)
    del genes[:int(n/3)]

In [7]:
pop = 16
genes = populate(pop)
for g in genes:
    print(g)

['0', 8, 0, 0.21, 0.0018, 8, 16, 'relu', -1]
['1', 8, 0, 0.18, 0.0005, 8, 128, 'relu', -1]
['2', 32, 0, 0.23, 0.0012, 8, 64, 'relu', -1]
['3', 32, 0, 0.32, 0.0008, 8, 16, 'relu', -1]
['4', 16, 0, 0.31, 0.001, 8, 64, 'relu', -1]
['5', 8, 0, 0.17, 0.0003, 8, 64, 'relu', -1]
['6', 8, 0, 0.22, 0.0018, 8, 16, 'relu', -1]
['7', 32, 0, 0.25, 0.0009, 8, 32, 'relu', -1]
['8', 32, 0, 0.16, 0.0009, 8, 32, 'relu', -1]
['9', 32, 0, 0.28, 0.0009, 8, 128, 'relu', -1]
['10', 16, 0, 0.21, 0.0005, 8, 32, 'relu', -1]
['11', 8, 0, 0.28, 0.0002, 8, 128, 'relu', -1]
['12', 8, 0, 0.26, 0.0002, 8, 64, 'relu', -1]
['13', 32, 0, 0.19, 0.0011, 8, 32, 'relu', -1]
['14', 16, 0, 0.12, 0.0005, 8, 64, 'relu', -1]
['15', 16, 0, 0.17, 0.0002, 8, 64, 'relu', -1]


In [8]:
acc = []
for i in range(len(genes)):
    genes[i][8] = (round(fitness(genes[i],indi),4))

Epoch 1/8
1339/1339 [==============================] - 18s 13ms/step - loss: 0.0510 - acc: 0.9842 - val_loss: 0.0204 - val_acc: 0.9921
Epoch 2/8
1339/1339 [==============================] - 11s 8ms/step - loss: 0.0195 - acc: 0.9926 - val_loss: 0.0062 - val_acc: 0.9980
Epoch 3/8
1339/1339 [==============================] - 10s 7ms/step - loss: 0.0128 - acc: 0.9954 - val_loss: 0.0055 - val_acc: 0.9980
Epoch 4/8
1339/1339 [==============================] - 10s 7ms/step - loss: 0.0113 - acc: 0.9961 - val_loss: 0.0066 - val_acc: 0.9976
Epoch 5/8
1339/1339 [==============================] - 10s 7ms/step - loss: 0.0095 - acc: 0.9967 - val_loss: 0.0067 - val_acc: 0.9977
Epoch 6/8
1339/1339 [==============================] - 10s 7ms/step - loss: 0.0095 - acc: 0.9969 - val_loss: 0.0058 - val_acc: 0.9978
Epoch 7/8
1339/1339 [==============================] - 10s 8ms/step - loss: 0.0080 - acc: 0.9974 - val_loss: 0.0017 - val_acc: 0.9994
Epoch 8/8
1339/1339 [==============================] - 10s 8m

In [9]:
genes.sort(key = lambda x: x[8])
for g in genes:
    print (g)

['6', 8, 0, 0.22, 0.0018, 8, 16, 'relu', 3.1007]
['4', 16, 0, 0.31, 0.001, 8, 64, 'relu', 61.0521]
['11', 8, 0, 0.28, 0.0002, 8, 128, 'relu', 85.0568]
['10', 16, 0, 0.21, 0.0005, 8, 32, 'relu', 91.8292]
['5', 8, 0, 0.17, 0.0003, 8, 64, 'relu', 92.0698]
['3', 32, 0, 0.32, 0.0008, 8, 16, 'relu', 92.1782]
['8', 32, 0, 0.16, 0.0009, 8, 32, 'relu', 93.5633]
['12', 8, 0, 0.26, 0.0002, 8, 64, 'relu', 93.8277]
['0', 8, 0, 0.21, 0.0018, 8, 16, 'relu', 93.8329]
['1', 8, 0, 0.18, 0.0005, 8, 128, 'relu', 93.8832]
['7', 32, 0, 0.25, 0.0009, 8, 32, 'relu', 94.8031]
['13', 32, 0, 0.19, 0.0011, 8, 32, 'relu', 95.1494]
['14', 16, 0, 0.12, 0.0005, 8, 64, 'relu', 95.7573]
['15', 16, 0, 0.17, 0.0002, 8, 64, 'relu', 95.9344]
['9', 32, 0, 0.28, 0.0009, 8, 128, 'relu', 96.0904]
['2', 32, 0, 0.23, 0.0012, 8, 64, 'relu', 97.013]


In [10]:
mutation(genes)

In [11]:
for g in genes:
    print(g)

['6', 8, 0, 0.22, 0.0018, 8, 16, 'relu', 3.1007]
['4', 16, 0, 0.31, 0.001, 8, 64, 'relu', 61.0521]
['11', 8, 0, 0.28, 0.0002, 8, 128, 'relu', 85.0568]
['10', 16, 0, 0.21, 0.0005, 8, 32, 'relu', 91.8292]
['5', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 92.0698]
['3', 32, 0, 0.32, 0.0008, 8, 16, 'relu', 92.1782]
['8', 32, 0, 0.16, 0.0009, 8, 32, 'relu', 93.5633]
['12', 8, 0, 0.26, 0.0005, 8, 64, 'relu', 93.8277]
['0', 8, 0, 0.21, 0.0018, 8, 16, 'relu', 93.8329]
['1', 8, 0, 0.18, 0.0005, 8, 128, 'relu', 93.8832]
['7', 32, 0, 0.25, 0.0009, 8, 32, 'relu', 94.8031]
['13', 32, 0, 0.19, 0.0011, 8, 32, 'relu', 95.1494]
['14', 16, 0, 0.12, 0.0005, 8, 64, 'relu', 95.7573]
['15', 16, 0, 0.17, 0.0002, 8, 64, 'relu', 95.9344]
['9', 32, 0, 0.28, 0.0009, 8, 128, 'relu', 96.0904]
['2', 32, 0, 0.23, 0.0012, 8, 64, 'relu', 97.013]


In [12]:
tot = 5
Gen = []
gen_best = []
for _ in range(tot):
    pop_size = len(genes)
    print(pop_size)
    keras.backend.clear_session()
    for i in range(len(genes)):
        genes[i][8]=fitness(genes[i],indi)
    genes.sort(key = lambda x: x[8])
    gen_best.append(genes[len(genes)-1])
    Gen.append(genes)
    natural_selection(genes)
    cross_over(genes)
    mutation(genes)

16
Epoch 1/8
1339/1339 [==============================] - 11s 8ms/step - loss: 0.0560 - acc: 0.9834 - val_loss: 0.0210 - val_acc: 0.9921
Epoch 2/8
1339/1339 [==============================] - 9s 7ms/step - loss: 0.0233 - acc: 0.9912 - val_loss: 0.0115 - val_acc: 0.9955
Epoch 3/8
1339/1339 [==============================] - 9s 7ms/step - loss: 0.0156 - acc: 0.9942 - val_loss: 0.0095 - val_acc: 0.9963
Epoch 4/8
1339/1339 [==============================] - 10s 7ms/step - loss: 0.0127 - acc: 0.9952 - val_loss: 0.0040 - val_acc: 0.9985
Epoch 5/8
1339/1339 [==============================] - 9s 7ms/step - loss: 0.0111 - acc: 0.9959 - val_loss: 0.0177 - val_acc: 0.9941
Epoch 6/8
1339/1339 [==============================] - 10s 7ms/step - loss: 0.0109 - acc: 0.9961 - val_loss: 0.0075 - val_acc: 0.9971
Epoch 7/8
1339/1339 [==============================] - 10s 7ms/step - loss: 0.0101 - acc: 0.9964 - val_loss: 0.0089 - val_acc: 0.9966
Epoch 8/8
1339/1339 [==============================] - 10s 7ms

In [13]:
genes.sort(key = lambda x: x[8])
for g in genes:
    print(g)

['9945', 32, 0, 0.21, 0.0009, 8, 128, 'relu', 94.88501189532118]
['9945', 32, 0, 0.21, 0.0009, 8, 128, 'relu', 94.88501189532118]
['9945', 32, 0, 0.21, 0.0009, 8, 128, 'relu', 94.88501189532118]
['1457173', 32, 0, 0.12, 0.0005, 8, 32, 'relu', 95.65952947396246]
['1457173', 32, 0, 0.12, 0.0005, 8, 32, 'relu', 95.65952947396246]
['1457173', 32, 0, 0.12, 0.0005, 8, 32, 'relu', 95.65952947396246]
['103', 16, 0, 0.21, 0.0005, 8, 32, 'relu', 95.72561459159398]
['103', 16, 0, 0.21, 0.0005, 8, 32, 'relu', 95.72561459159398]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]


In [14]:
for i in gen_best:
    print(i)

['7', 32, 0, 0.25, 0.0009, 8, 32, 'relu', 91.37721385144066]
['103', 16, 0, 0.21, 0.0005, 8, 32, 'relu', 95.72561459159398]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]
['9945', 32, 0, 0.21, 0.0009, 8, 128, 'relu', 94.88501189532118]
['1457173', 32, 0, 0.12, 0.0005, 8, 32, 'relu', 95.65952947396246]


In [15]:
for g in Gen:
    for gg in g:
        print(gg)
    print("----------------------------------------")

['9945', 32, 0, 0.21, 0.0009, 8, 128, 'relu', 94.88501189532118]
['9945', 32, 0, 0.21, 0.0009, 8, 128, 'relu', 94.88501189532118]
['9945', 32, 0, 0.21, 0.0009, 8, 128, 'relu', 94.88501189532118]
['1457173', 32, 0, 0.12, 0.0005, 8, 32, 'relu', 95.65952947396246]
['1457173', 32, 0, 0.12, 0.0005, 8, 32, 'relu', 95.65952947396246]
['1457173', 32, 0, 0.12, 0.0005, 8, 32, 'relu', 95.65952947396246]
['103', 16, 0, 0.21, 0.0005, 8, 32, 'relu', 95.72561459159398]
['103', 16, 0, 0.21, 0.0005, 8, 32, 'relu', 95.72561459159398]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]
['354166', 8, 0, 0.17, 0.0005, 8, 64, 'relu', 96.4842717420037]
----------------------------------------
['9945', 32, 0, 0.21, 0.0009, 8, 128, 'relu', 94.8850